In [2]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import missingno as msno

In [9]:
# Dummy Features
# Preparing Features for Analysis: Dummy-Coding, Imputation, Recombination

def create_dummies(filename, ID = False):
    class_data = pd.read_csv(filename)

    # Separating numeric and categorical variables for Dummy-coding:
    dummy_vars = ['day_of_week', 'is_weekend', 'max_screen_duration_day_part']
    dummyfeatures = class_data[dummy_vars]
    dummyfeatures = dummyfeatures.applymap(str)  # Converting to string to account for categorical AZ variables

    # Dummy-coding categorical variables:
    dummyfeatures = pd.get_dummies(dummyfeatures, dtype = float)

    # Recombine both dfs and define target variable
    if ID: # if we want to keep the ID column
        all_features = pd.concat([class_data[[col for col in class_data.columns if col not in dummy_vars]], dummyfeatures], axis = 1)
        all_features.to_csv('data/final_RNN.csv', index=False)
    else:
        all_features = pd.concat([class_data[[col for col in class_data.columns if col not in dummy_vars]], dummyfeatures], axis = 1).iloc[:,2:]
        all_features.to_csv('data/final.csv', index = False)

create_dummies('data/imputed_data_v2.csv', ID = True)

In [26]:
# RNN

data_rnn = pd.read_csv('data/final_RNN.csv').iloc[:,1:]

g = data_rnn.groupby('id').cumcount()
L = (data_rnn.set_index(['id',g])
       .unstack(fill_value=0)
       .stack().groupby(level=0)
       .apply(lambda x: x.values.tolist())
       .tolist())

tensor_rnn = np.array(L)

In [23]:
## Torch Setup

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
37,2014-04-29,4,6.6,0.0974598031302916,0.6,0.4,4524.966001987459,0.0,0.0,1348.7640000000004,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
38,2014-04-30,4,6.2,0.0840277777777778,-0.2,0.2,7556.272999286652,0.0,0.0,1069.053,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
39,2014-05-01,5,6.8,0.1168242296918767,0.0,0.4,5536.597999811173,5.0,0.0,2948.324,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
40,2014-05-02,5,7.2,0.1430287453399221,0.2,0.8,4744.676000833512,0.0,0.0,733.06,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
41,2014-05-03,5,7.8,0.0464574820655356,0.0,0.8,3909.751001358033,3.0,0.0,1215.653,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
42,2014-05-04,5,7.4,0.0175177860999472,-0.2,0.4,4421.830999612806,0.0,0.0,418.308,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
43,2014-05-06,5,6.5,0.0501412429378531,0.25,0.25,6861.393000602726,1.0,1.0,890.806,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
44,2014-05-07,5,6.8,0.0913150305099551,0.0,0.4,2567.527999877929,2.0,0.0,667.753,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
45,2014-05-08,5,6.6,0.1056320621468926,-0.2,0.4,9094.6829996109,0.0,0.0,13109.071,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
46,2014-05-09,5,6.8,0.1131502812989602,0.2,0.6,8174.018999099728,0.0,0.0,18385.732000000004,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# 60 input features
# 67 sequence length


# to tune: 
# Number hidden layers
# Hidden size
# tanh vs. relu
# Bias
# Dropout
# Learning Rate
# 

class Mood_RNN(nn.Module):
    def __init__(self, num_layers, hidden_size, batch_size):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.num_features = 65
        self.num_classes = 10
        self.rnn = nn.RNN(self.num_features, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, self.num_classes)
        
    def forward(self, x, hidden_state):
        
        output, new_hidden = self.rnn(x, hidden_state) # run RNN on input and hidden state
        output_rnn = output.view(-1, self.batch_size, self.hidden_size) # reshape output to (whatever, hidden_size)
        output_fc = self.fc(output_rnn) # run RNN output through fully connected layer
        
        return output_fc, new_hidden

    
    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size)
    

mood_RNN = Mood_RNN(5, 10)

## Transforming input from input[x,y,z] to input[y,x,z]:
tensor_rnn = np.transpose(tensor_rnn, (1,0,2))

for i in range(0, tensor_rnn.shape[1], 3):
    print(i)
    
## Check out GRU as well

In [25]:
tensor_rnn.shape

(67, 27, 60)

In [24]:
tensor_rnn = np.transpose(tensor_rnn, (1,0,2))

for i in range(0, tensor_rnn.shape[1], 3):
    rnn_input = tensor_rnn[:,i:i+3,:]
    print(i)

0
3
6
9
12
15
18
21
24


In [ ]:
input_data = 

In [ ]:
# Hyperparameter Tuning: Question 11

learning_rate = [0.0005, 0.001, 0.01]
momentum = [0.9, 0.95]


###Define the grid of parameters to search
hyper_grid = {'learning_rate': learning_rate,
              'momentum': momentum}

hpts = ParameterGrid(hyper_grid)

results_dict_tune = {}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for param_run, params in enumerate(hpts):
    
    lr = params['learning_rate']
    mom = params['momentum']
    net = Net()
    net.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=mom)

    loss_list = list()

    for epoch in range(10): 

        for i, data in tqdm(enumerate(trainloader, 0)):
            # get the inputs; data is a list of [inputs, labels]
            
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss_list.append(loss.item())
            loss.backward()
            optimizer.step()

            # print statistics
    
    correct_train = 0
    total_train = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in trainloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
            
    train_acc = correct_train / total_train
    
    correct_val = 0
    total_val = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    
    val_acc = correct_val / total_val

    print(f'Training Accuracy with lr {lr} and mom {mom}: {train_acc}')
    print(f'Validation Accuracy with lr {lr} and mom {mom}: {val_acc}')

    results_dict_tune[param_run] = [train_acc, val_acc, loss_list]

print('Finished Training')

with open('/content/gdrive/My Drive/tuning_results.pkl', 'wb') as f:
    pickle.dump(results_dict_tune, f)